# Finetuning with hugging face 

In [2]:
import tensorflow as tf

from training import keras_train
from pascal import get_pascal_tfds
from datasets import load_dataset
from transformers import ViTForImageClassification

2023-04-25 15:59:14.662669: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 15:59:15.173910: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-04-25 15:59:15.173963: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-04-25 15:59:15.173981: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [ ]:

image_size = 224
seq_len = 1024  # 32 * 32 for (image_size=512)//16 patches
d_model = 512
batch_size = 16

ds = load_dataset("beans")
example = ds["train"][400]
image = example["image"]
labels = ds["train"].features["labels"]
labels = ds["train"].features["labels"].names

model_name_or_path = "google/vit-base-patch16-224-in21k"
model = ViTForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),  # model creates classification head with correct number of classes
    id2label={str(i): c for i, c in enumerate(labels)},  # mapping between human readable labels and model's internal labels
    label2id={c: str(i) for i, c in enumerate(labels)},
)

Found cached dataset beans (/home/fergus/.cache/huggingface/datasets/beans/default/0.0.0/90c755fb6db1c0ccdad02e897a37969dbf070bed3755d4391e269ff70642d791)
100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 135.53it/s]
Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTForImageClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTForImageClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTForImageClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTForImageClassification were not initialize

In [ ]:
# model.state_dict() dict of layers to weights

In [ ]:
import torch.nn as nn

# for param in self.vit_model.parameters():
#     param.requires_grad = False
    
    
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.vit_model = model.vit.encoder
        self.classifier = nn.Linear(768, 20)

    def forward(self, x):
        hidden_states = self.vit_model(x).last_hidden_state
        out = self.classifier(hidden_states[:, 0])
        return out
    
my_model = MyModel()

In [8]:
model.vit.last_hidden_state

AttributeError: 'ViTModel' object has no attribute 'last_hidden_state'

In [53]:
from pascal import get_pascal_tfds 

def image_sequentialisation(image, label):
    """
    # TODO : batch dim?
    Convert image into sequence: f : R^(H x W x 3) -> R^(n x -1) # n flattened patches
    L = HW / 256

    So essentially divid image into 16x16 grid. -> input should be dividsible by 16

    """
    # start with loopy implementation
    steps = image_size // 16
    # patches = tf.zeros((batch_size, steps * steps, 16 * 16 * 3))
    patches = []
    for i in range(steps):
        for j in range(steps):
            patch = image[:, i * 16 : (i + 1) * 16, j * 16 : (j + 1) * 16, :]
            flatten_patch = tf.reshape(patch, shape=(batch_size, -1))
            patches.append(flatten_patch)
    patches = tf.stack(patches)
    sequence = tf.reshape(patches, shape=(batch_size, steps * steps, 16 * 16 * 3))
    return sequence, label


train_tfds, val_tfds = get_pascal_tfds(image_size=image_size, batch_size=batch_size)
train_tfds = train_tfds.map(image_sequentialisation)
val_tfds = val_tfds.map(image_sequentialisation)

In [67]:
import tensorflow as tf
import torch

class TFToPyTorchDataset(torch.utils.data.Dataset):
    def __init__(self, tf_dataset):
        self.tf_dataset = tf_dataset

    def __getitem__(self, index):
        # Get a single example from the TensorFlow dataset
        example = next(iter(self.tf_dataset.skip(index).take(1)))
        
        # Convert the example from TensorFlow tensor to PyTorch tensor
        x = torch.from_numpy(example[0].numpy())
        y = torch.from_numpy(example[1].numpy())

        return x, y

    def __len__(self):
        return len(self.tf_dataset)
    
batch_size = 8
    
pytorch_dataset_train = TFToPyTorchDataset(train_tfds)
pytorch_dataloader_train = torch.utils.data.DataLoader(pytorch_dataset_train, batch_size=batch_size)
#pytorch_dataset_test = TFToPyTorchDataset(test_tfds)
#pytorch_dataloader_test = torch.utils.data.DataLoader(pytorch_dataset_test, batch_size=batch_size)

In [57]:
pytorch_dataloader

In [59]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.vit_model = model.vit.encoder
        self.classifier = nn.Linear(768, 20)

    def forward(self, x):
        hidden_states = self.vit_model(x).last_hidden_state
        print(hidden_states.shape)
        out = self.classifier(hidden_states[:, 0])
        return out

In [88]:

class SETR(nn.Module):
    def __init__(self, vit, n_classes):
        super().__init__()
        self.n_classes = n_classes
        self.vit_model = vit.vit.encoder
        self.decoder_conv2d_1 = nn.Conv2d(in_channels=768, out_channels=256, kernel_size=(1,1))
        self.decoder_conv2d_2 = nn.Conv2d(in_channels=256, out_channels=n_classes, kernel_size=(1,1))
        self.bn = nn.BatchNorm2d(num_features=256)
        self.relu = nn.ReLU()
        self.upsample = nn.UpsamplingBilinear2d(size=(16,16))

    def decoder(self, encoder_outputs):
        print(encoder_outputs.shape)
        # x = torch.reshape(encoder_outputs, shape=(batch_size, image_size // 16, image_size // 16, d_model))
        x = torch.reshape(encoder_outputs, shape=(batch_size, d_model, 14, 14))
        x = self.decoder_conv2d_1(x)
        x = self.bn(x)
        x = self.relu(x)
        x = self.decoder_conv2d_2(x)
        x = self.upsample(x)
        return x 

    def forward(self, x):
        hidden_states = self.vit_model(x).last_hidden_state
        x = self.decoder(hidden_states)
        x = nn.Softmax(x)
        return x

    
image_size= 512
d_model=768
    
setr = SETR(model, 21)

for batch_ndx, sample in enumerate(pytorch_dataset_train):
    print(sample[0].shape)
    setr(sample[0])
    
    

there are 964 training examples
torch.Size([16, 196, 768])
torch.Size([16, 196, 768])
there are 964 training examples
torch.Size([16, 196, 768])
torch.Size([16, 196, 768])
there are 964 training examples
torch.Size([16, 196, 768])
torch.Size([16, 196, 768])
there are 964 training examples


KeyboardInterrupt: 

# Training 

In [91]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [94]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

def train(dataloader, model, loss_fn, optimizer):
    size = 99
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
setr = setr.to(device)
train(pytorch_dataset_train, setr, loss_fn, optimizer)

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 23.68 GiB total capacity; 352.89 MiB already allocated; 15.31 MiB free; 388.00 MiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [95]:
%%bash
nvidia-smi

Tue Apr 25 13:34:40 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:03:00.0 Off |                  Off |
|  0%   52C    P8    35W / 450W |  24232MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------